## Comparison with APBS and MIBPB

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from bempp_pbs.postprocess import PLOT_PARAMS, get_df

In [2]:
def richardson_extrapolation(f1, f2, f3):
    return (f1*f3 - f2**2) / (f3 - 2*f2 + f1)

def order_of_convergence(f1, f2, f3, r=2):
    return np.log((f3-f2)/(f2-f1)) / np.log(r)

In [3]:
def parse_mibpb(file):
    """ Given an output file from MIBPB, parse and return the solvation energy. """
    with open(file) as f:
        for line in f.readlines():
            line = line.strip()
            if line.startswith('Electrostatics solvation'):
                e_solv = float(line.split(':')[-1])
            if line.startswith('grid dimension'):
                grid_size = line.split(':')[-1].strip()
            if line.startswith('The total number of atoms is:'):
                n_atoms = int(line.split(':')[-1])
    return e_solv, grid_size, n_atoms

We selected the following 9 proteins to compare Bempp' results with those from APBS and MIBPB.

In [4]:
molecule_ids = ['1AJJ', '1VJW', '5PTI', '1R69', '1A2S', '1SVR', '1A63', '1A7M', '1F6W']  # ordered by n_atoms

### Load APBS results

In [5]:
apbs_df = pd.read_csv('../runs/APBS_result/results.txt', delim_whitespace=True, index_col=0)
# convert kJ/mol to kcal/mol
apbs_df.dG1 /= 4.184
apbs_df.dG2 /= 4.184
apbs_df.dG3 /= 4.184
apbs_exact = richardson_extrapolation(apbs_df.dG3, apbs_df.dG2, apbs_df.dG1).values
apbs_order = order_of_convergence(apbs_df.dG3, apbs_df.dG2, apbs_df.dG1).values

apbs_df['apbs_exact e_solv[kcal/mol]'] = apbs_exact
apbs_df['apbs_order_convergence'] = apbs_order
#apbs_df['apbs_finest e_solv[kcal/mol]'] = apbs_df.dG3
#apbs_df.drop(columns=['N1^3', 'N2^3', 'N3^3', 'dG1', 'dG2', 'dG3'], inplace=True)
#apbs_df

compute relative error w.r.t the extrapolated value

In [6]:
apbs_df['apbs_error1'] = np.abs((apbs_df.dG1 - apbs_exact) / apbs_exact)
apbs_df['apbs_error2'] = np.abs((apbs_df.dG2 - apbs_exact) / apbs_exact)
apbs_df['apbs_error3'] = np.abs((apbs_df.dG3 - apbs_exact) / apbs_exact)

In [7]:
apbs_df

,N1^3,N2^3,N3^3,dG1,dG2,dG3,apbs_exact e_solv[kcal/mol],apbs_order_convergence,apbs_error1,apbs_error2,apbs_error3
Mol,,,,,,,,,,,
1A2S,65,129,257,-480.533236,-467.519359,-461.569753,-456.558928,1.129185,0.052511,0.024007,0.010975
1A63,129,257,513,-584.612888,-569.896418,-563.767329,-559.392803,1.263687,0.045085,0.018777,0.007820
1A7M,129,257,513,-545.943830,-534.134153,-528.766337,-524.293491,1.137562,0.041294,0.018769,0.008531
1AJJ,65,129,257,-277.849170,-269.676310,-267.211002,-266.146143,1.729074,0.043972,0.013264,0.004001
1F6W,129,257,513,-1359.965514,-1313.040799,-1292.819151,-1277.505740,1.214447,0.064547,0.027816,0.011987
1R69,97,193,385,-272.268031,-266.494940,-263.685216,-261.021168,1.038915,0.043088,0.020971,0.010206
1SVR,97,193,385,-414.208395,-402.155386,-397.100372,-393.448873,1.253606,0.052763,0.022129,0.009281
1VJW,97,193,385,-306.237636,-301.358332,-299.073922,-297.062845,1.094853,0.030885,0.014460,0.006770
5PTI,97,193,385,-323.167233,-315.981778,-313.296731,-311.694770,1.420132,0.036807,0.013754,0.005140


### Parse Bempp and MIBPB results from raw output

In [8]:
bempp_result = list()
mibpb_result = list()
bempp_mesh = list()
mibpb_fine_grid_size = list()
n_atoms = list()
for molecule in molecule_ids:
    bempp_path = f"../runs/{molecule}_convergence/derivative_ex"  # bempp result
    df = get_df(bempp_path, formulation='derivative', skip4=True)
    if molecule=='5PTI':       # we have 5 meshes for 5PTI, only need the last three
        bempp_result.append(df['e_solv [kcal/Mol]'].to_list()[-3:])
        bempp_mesh.append(df.index.values[-3:])
    else:
        bempp_result.append(df['e_solv [kcal/Mol]'].to_list())
        bempp_mesh.append(df.index.values)
    
    for h in ['1.00', '0.50', '0.25']:
        mibpb_path = f"../runs/MIBPB_result/{molecule}/{molecule}_h{h}.dat"
        e_solv, grid_size, _n_atoms = parse_mibpb(mibpb_path)
        mibpb_result.append(e_solv)
        if h == '0.25':
            mibpb_fine_grid_size.append(grid_size)
            n_atoms.append(_n_atoms)
bempp_result = np.array((bempp_result))
mibpb_result = np.array(mibpb_result).reshape((-1,3))
bempp_mesh = np.array(bempp_mesh, dtype=int)

In [9]:
bempp_result

array([[ -276.0496385 ,  -271.68437065,  -269.90059992],
       [ -317.39408852,  -309.25250871,  -305.56092084],
       [ -330.44014755,  -321.33078237,  -317.37397301],
       [ -278.51111653,  -271.22504491,  -267.87519922],
       [ -480.6756686 ,  -470.15975014,  -465.33635441],
       [ -424.68076421,  -410.36077241,  -403.97187   ],
       [ -600.24792595,  -581.98679606,  -573.82691627],
       [ -557.66663012,  -543.40245032,  -536.90871768],
       [-1355.48785407, -1324.51172387, -1311.170178  ]])

In [10]:
bempp_exact = richardson_extrapolation(bempp_result[:,2], bempp_result[:,1], bempp_result[:,0])
bempp_order = order_of_convergence(bempp_result[:,2], bempp_result[:,1], bempp_result[:,0])
bempp_error = (bempp_result - bempp_exact.reshape((-1,1))) / bempp_exact.reshape((-1,1))

In [11]:
bempp_df = pd.DataFrame.from_dict({
    'id': molecule_ids,
    'n_atoms': n_atoms,
    'bempp_order_convergence': bempp_order,
    'bempp_exact e_solv[kcal/mol]': bempp_exact,
    'bempp_error1': bempp_error[:,0],
    'bempp_error2': bempp_error[:,1],
    'bempp_error3': bempp_error[:,2],
    'bempp_coarse': bempp_mesh[:,0],
    'bempp_medium': bempp_mesh[:,1],
    'bempp_fine': bempp_mesh[:,2]
})
bempp_df.set_index('id', inplace=True)
bempp_df

,n_atoms,bempp_order_convergence,bempp_exact e_solv[kcal/mol],bempp_error1,bempp_error2,bempp_error3,bempp_coarse,bempp_medium,bempp_fine
id,,,,,,,,,
1AJJ,513,1.291140,-268.668045,0.027475,0.011227,0.004588,8544,17200,34468
1VJW,826,1.141067,-302.498484,0.049242,0.022327,0.010124,11544,23296,46660
5PTI,892,1.203013,-314.335415,0.051234,0.022254,0.009667,12512,25204,50596
1R69,997,1.121047,-265.024381,0.050889,0.023397,0.010757,12004,24216,48648
1A2S,1272,1.124454,-461.249388,0.042117,0.019318,0.008861,17784,35732,71648
1SVR,1433,1.164391,-398.825279,0.064829,0.028924,0.012904,18644,37348,75004
1A63,2065,1.162156,-567.235293,0.058199,0.026006,0.011621,27996,56356,113124
1A7M,2804,1.135277,-531.481930,0.049267,0.022429,0.010211,30576,61652,123776
1F6W,8247,1.215231,-1301.076555,0.041820,0.018012,0.007758,77464,156140,313692


## Merge APBS and Bempp results

In [13]:
apbs_bempp = pd.merge(bempp_df, apbs_df, left_index=True, right_index=True)
apbs_bempp.drop(inplace=True, columns=['dG1', 'dG2', 'dG3'])
apbs_bempp

,n_atoms,bempp_order_convergence,bempp_exact e_solv[kcal/mol],bempp_error1,bempp_error2,bempp_error3,bempp_coarse,bempp_medium,bempp_fine,N1^3,N2^3,N3^3,apbs_exact e_solv[kcal/mol],apbs_order_convergence,apbs_error1,apbs_error2,apbs_error3
1AJJ,513,1.291140,-268.668045,0.027475,0.011227,0.004588,8544,17200,34468,65,129,257,-266.146143,1.729074,0.043972,0.013264,0.004001
1VJW,826,1.141067,-302.498484,0.049242,0.022327,0.010124,11544,23296,46660,97,193,385,-297.062845,1.094853,0.030885,0.014460,0.006770
5PTI,892,1.203013,-314.335415,0.051234,0.022254,0.009667,12512,25204,50596,97,193,385,-311.694770,1.420132,0.036807,0.013754,0.005140
1R69,997,1.121047,-265.024381,0.050889,0.023397,0.010757,12004,24216,48648,97,193,385,-261.021168,1.038915,0.043088,0.020971,0.010206
1A2S,1272,1.124454,-461.249388,0.042117,0.019318,0.008861,17784,35732,71648,65,129,257,-456.558928,1.129185,0.052511,0.024007,0.010975
1SVR,1433,1.164391,-398.825279,0.064829,0.028924,0.012904,18644,37348,75004,97,193,385,-393.448873,1.253606,0.052763,0.022129,0.009281
1A63,2065,1.162156,-567.235293,0.058199,0.026006,0.011621,27996,56356,113124,129,257,513,-559.392803,1.263687,0.045085,0.018777,0.007820
1A7M,2804,1.135277,-531.481930,0.049267,0.022429,0.010211,30576,61652,123776,129,257,513,-524.293491,1.137562,0.041294,0.018769,0.008531
1F6W,8247,1.215231,-1301.076555,0.041820,0.018012,0.007758,77464,156140,313692,129,257,513,-1277.505740,1.214447,0.064547,0.027816,0.011987


#### generate table for mesh info 

In [14]:
grid_cols = ['N1^3', 'N2^3', 'N3^3', 'bempp_coarse', 'bempp_medium', 'bempp_fine']
apbs_bempp[grid_cols]

,N1^3,N2^3,N3^3,bempp_coarse,bempp_medium,bempp_fine
1AJJ,65,129,257,8544,17200,34468
1VJW,97,193,385,11544,23296,46660
5PTI,97,193,385,12512,25204,50596
1R69,97,193,385,12004,24216,48648
1A2S,65,129,257,17784,35732,71648
1SVR,97,193,385,18644,37348,75004
1A63,129,257,513,27996,56356,113124
1A7M,129,257,513,30576,61652,123776
1F6W,129,257,513,77464,156140,313692


#### generate table of convergence results from APBS and Bempp

In [15]:
apbs_cols = ['apbs_error1', 'apbs_error2', 'apbs_error3', 'apbs_exact e_solv[kcal/mol]', 'apbs_order_convergence']
bempp_cols = [ col.replace('apbs', 'bempp') for col in apbs_cols]
cols_order = apbs_cols + bempp_cols
apbs_bempp[cols_order]
apbs_bempp['rel_diff'] = np.abs((apbs_bempp['bempp_exact e_solv[kcal/mol]']-apbs_bempp['apbs_exact e_solv[kcal/mol]'])/apbs_bempp['apbs_exact e_solv[kcal/mol]'])

In [16]:
apbs_bempp[cols_order]
# apbs_bempp[cols_order].to_csv('apbs_result', float_format='%.2e')

,apbs_error1,apbs_error2,apbs_error3,apbs_exact e_solv[kcal/mol],apbs_order_convergence,bempp_error1,bempp_error2,bempp_error3,bempp_exact e_solv[kcal/mol],bempp_order_convergence
1AJJ,0.043972,0.013264,0.004001,-266.146143,1.729074,0.027475,0.011227,0.004588,-268.668045,1.291140
1VJW,0.030885,0.014460,0.006770,-297.062845,1.094853,0.049242,0.022327,0.010124,-302.498484,1.141067
5PTI,0.036807,0.013754,0.005140,-311.694770,1.420132,0.051234,0.022254,0.009667,-314.335415,1.203013
1R69,0.043088,0.020971,0.010206,-261.021168,1.038915,0.050889,0.023397,0.010757,-265.024381,1.121047
1A2S,0.052511,0.024007,0.010975,-456.558928,1.129185,0.042117,0.019318,0.008861,-461.249388,1.124454
1SVR,0.052763,0.022129,0.009281,-393.448873,1.253606,0.064829,0.028924,0.012904,-398.825279,1.164391
1A63,0.045085,0.018777,0.007820,-559.392803,1.263687,0.058199,0.026006,0.011621,-567.235293,1.162156
1A7M,0.041294,0.018769,0.008531,-524.293491,1.137562,0.049267,0.022429,0.010211,-531.481930,1.135277
1F6W,0.064547,0.027816,0.011987,-1277.505740,1.214447,0.041820,0.018012,0.007758,-1301.076555,1.215231


#### relative difference of the extrapolated solvation energy

In [17]:
np.abs((apbs_bempp['apbs_exact e_solv[kcal/mol]']-apbs_bempp['bempp_exact e_solv[kcal/mol]'])/(apbs_bempp['apbs_exact e_solv[kcal/mol]']))

1AJJ    0.009476
1VJW    0.018298
5PTI    0.008472
1R69    0.015337
1A2S    0.010274
1SVR    0.013665
1A63    0.014020
1A7M    0.013711
1F6W    0.018451
dtype: float64

-----
## Compare with MIBPB results

In [18]:
# mibpb_exact = richardson_extrapolation(mibpb_result[:,2], mibpb_result[:,1], mibpb_result[:,0])
# mibpb_order = order_of_convergence(mibpb_result[:,2], mibpb_result[:,1], mibpb_result[:,0])
mibpb_result

array([[ -271.2630376 ,  -271.63665356,  -271.58766711],
       [ -306.69930751,  -307.68319052,  -307.71479371],
       [ -319.20647378,  -317.31604679,  -317.5423135 ],
       [ -271.0588638 ,  -270.99250176,  -270.93616902],
       [ -462.01139998,  -462.85904724,  -463.11131482],
       [ -409.66220052,  -410.04988387,  -410.39884182],
       [ -582.69168199,  -583.53638064,  -581.02207921],
       [ -539.38441171,  -541.05630814,  -531.92905946],
       [-1338.42873523, -1337.08972471, -1336.47998636]])

In [19]:
mibpb_bempp = pd.DataFrame.from_dict({
    'id': molecule_ids,
    'mibpb_fine_grid': mibpb_fine_grid_size,
    'mibpb_finest e_solv[kcal/mol]': mibpb_result[:,2],
    'bempp_finest e_solv[kcal/mol]': bempp_result[:,2]
    #'rel_diff_exact_mibpb': np.abs((bempp_exact-mibpb_exact)/mibpb_exact),
    #'bempp_finest e_solv[kcal/mol]': bempp_result[:,2],
    #'mibpb_finest e_solv[kcal/mol]': mibpb_result[:,2],
    #'rel_diff_finest_mibpb': np.abs((bempp_result[:,2]-mibpb_result[:,2])/mibpb_result[:,2])
})
mibpb_bempp.set_index('id', inplace=True)

In [20]:
mibpb_bempp.round(2)

,mibpb_fine_grid,mibpb_finest e_solv[kcal/mol],bempp_finest e_solv[kcal/mol]
id,,,
1AJJ,132 142 154,-271.59,-269.90
1VJW,137 160 148,-307.71,-305.56
5PTI,145 158 178,-317.54,-317.37
1R69,167 155 148,-270.94,-267.88
1A2S,173 166 186,-463.11,-465.34
1SVR,174 187 192,-410.40,-403.97
1A63,283 184 186,-581.02,-573.83
1A7M,191 266 225,-531.93,-536.91
1F6W,330 272 309,-1336.48,-1311.17


#### relative difference of the solvation energy computed from the finest mesh

In [21]:
diff = np.abs((mibpb_bempp['mibpb_finest e_solv[kcal/mol]']-mibpb_bempp['bempp_finest e_solv[kcal/mol]'])/(mibpb_bempp['mibpb_finest e_solv[kcal/mol]']))
diff

id
1AJJ    0.006212
1VJW    0.007000
5PTI    0.000530
1R69    0.011298
1A2S    0.004805
1SVR    0.015660
1A63    0.012384
1A7M    0.009362
1F6W    0.018938
dtype: float64